Siamese Networks are a type of neural networks to decide if the elements in an input pair are similar or not. In Siamese Networks, there are to **identical (same weights and architecture)** neural network that process the inputs separately. Similarity of their outputs is measured with cosine similarity and they accepted as similar, if cosine similarity is a above a parameter $\tau$.

An example Siamese Network for duplicate question detection can be constructed as follows:

<img src='images/siamese.png' style='zoom: 40%' />

We refer to Question 1 in the figure as `Anchor` sentence and Question 2 as `Positive` if it is similar with Q1 and `Negative` if it is dissimilar. Siamese networks aim to maximize the similarity between similar pairs and minimize the similarity between dissimilar ones. We can define a triplet cost function (a cost function with three inputs) as follows to enable such learning.

$\mathcal{L}(A,P,N) = diff = s(A, N) - s(A, P) \tag{1}$

where, $A$ denotes anchor representation and $N$ and $P$ denote the vector of its negative and positive pair, respectively. $s(x, y)$ is a similarity function such as cosine similarity. To minimize $\mathcal{L}$, the model should maximize $s(A,P)$, the similarity between similar sentences, and minimize $s(A,N)$, the similarity between dissimilar sentences, as desired.

Though $\mathcal{L}$ seems to achieve the goal, it has a major drawback that it results in a negative loss, when positive pair has higher similarity and signals the model that it should update its weights. Yet, we do not want to update the model weights when they create accurate predictions. Therefore, we assume that an $\mathcal{L}$ below a threshold $\alpha$ is a correct prediction and modify the loss function to result in $0$ loss in this case.

$\mathcal{L}(A,P,N) = max\left(diff + \alpha, 0\right) \tag{2}$

where $diff$ is computed as in Eq. $1$. Now $\mathcal{L}(A,P,N) = 0$  when similarity difference is below $\alpha$. In other words, weights are not updated for accurate predictions.

In a typical scenario, we have similar sentence pairs in the dataset but no negative pairs. Though randomly sampling negative sentences is a feasible approach to create negative pairs, it makes minimizing the loss too easy, and thus do not teach much to the models. The difficult pairs should be selected for a better training.

We adopt a different apporach for negative pairs and do not sample them explicitly. Instead, we construct two training batches with some conditions and modify loss function to promote learning from more difficult pairs.

We organize the training batches such that (i) there is no duplicate sentence in the same batch and (ii) the sentences in the same indexed row of the two batches are duplicate. Then, we compute their vector representations through forward propagation and compute pairwise similarity of the vectors. The resulting matrix is expected to be similar to the one below, where diagonals have higher similarity scores, since they correspond to the similarities of duplicate sentences. The matrix below is a hypothetical one for a batch size of 4.

<img src='images/sim_matrix.png' style='zoom: 40%' />

Now we modify the loss function to decrease noise due to too many negative scores in the batch, and promote learning from difficult samples. To decrease the noise, we compute mean of the off-diagonal similarities in each row while computing $s(A, N)$ term in $diff$. This way, we decrease the effect of outliers and decrease noisy signals during training. Thus, we compute a loss as $\mathcal{L}_1(A, P, N) = max(mean\_neg - s(A,P)) + \alpha, 0)$

To promote difficult samples, we find the highest score in the off-diagonal entries that is also less than the diagonal. We interpret this score as the most difficult entry and compute a loss as $\mathcal{L}_2(A, P, N) = max(closest\_neg - s(A,P)) + \alpha, 0)$. Last, we combine these two losses:

$$
\begin{align}
\mathcal{L}_{full}(A, P, N) &= \mathcal{L}_1 + \mathcal{L}_2 \\
J(A, P, N) &= \sum_{i=1}^{m}  \mathcal{L}_{full}(A^{(i)}, P^{(i)}, N^{(i)}) \\
\end{align}
$$

**Remark:** During test time, we just compute the input representations and apply a rule based classification by comparing the similarity with the predefined threshold $\tau$.

Siamese networks are suitable models for one-shot learning, where you only have a labeled dataset with one sample per class. In such a scenario, training a multi-class classification model is infeasible since classes do not contain much sample. However, learning a similarity threshold $\tau$ to discriminate classes is possible with a siamese network.